In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import calendar as cld
import matplotlib.pyplot as plt
import matplotlib.colors
colors_land = plt.cm.terrain(np.linspace(0.25, 1, 256))
import proplot as pplt # New plot library (https://proplot.readthedocs.io/en/latest/)
pplt.rc['savefig.dpi'] = 300 # 1200 is too big! #https://proplot.readthedocs.io/en/latest/basics.html#Creating-figures
from scipy.stats import chi2
from numba import njit,prange
import matplotlib as mpl
mpl.rcParams['hatch.linewidth'] = 0.05  # previous pdf hatch linewidth
from matplotlib.dates import DateFormatter

In [3]:
# function for seasonal mean (from Nathan)
def season_mean(ds, calendar="standard"):
    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = ds.time.dt.days_in_month

    # Calculate the weights by grouping by 'time.season'
    weights = (
        month_length.groupby("time.season") / month_length.groupby("time.season").sum()
    )

    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weights.groupby("time.season").sum().values, np.ones(4))

    # Calculate the weighted average
    return (ds * weights).groupby("time.season").sum(dim="time")

# This method gives different weights to days from 28-, 30- and 31-day months, instead of giving an equal weight to each day in the season.

In [4]:
# function for seasonal mean number 2 (from Ian)
def season_mean2(ds, calendar="standard"):
    #make a DataArray with the season of each day
    seas_array = ds.time.dt.season
    
    # count days in winter season (varies with leap years)
    nb_DJF = seas_array.str.count('DJF').sum()
    
    wgt_DJF = 1/nb_DJF.values.item()
    wgt_MAM = 1/(31+30+31)
    wgt_JJA = 1/(30+31+31)
    wgt_SON = 1/(30+31+30)

    weight = seas_array.str.replace('DJF',str(wgt_DJF)).str.replace('MAM',str(wgt_MAM)).str.replace('JJA',str(wgt_JJA)).str.replace('SON',str(wgt_SON))
    weight = weight.astype('float')
    
    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weight.groupby("time.season").sum().values, np.ones(4))

    # Calculate the weighted average
    return (ds * weight).groupby("time.season").sum(dim="time")

In [10]:
first_year = 1902
last_year = 1910
for year in list(range(first_year,last_year+1)):
    print(year,end=' ')
    ds_T = xr.open_dataset('/bettik/beaumetj/MARout/MAR-ERA-20C/daily/ICE.ERA-20C_v1.EUf.TTz.'+str(year)+'.nc')
    seasonal_mean_year=season_mean(ds_T.TTz.sel(ztqlev=2)).sortby(xr.DataArray(['DJF','MAM','JJA', 'SON'],dims=['season']))
    if year> first_year:
        seasonal_mean += seasonal_mean_year
    else:
        seasonal_mean = xr.DataArray.copy(seasonal_mean_year)
seasonal_mean /= (last_year-first_year+1)

print(seasonal_mean.shape, seasonal_mean.season.data)

1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 (4, 126, 201) ['DJF' 'MAM' 'JJA' 'SON']


In [11]:
first_year = 1902
last_year = 1910
for year in list(range(first_year,last_year+1)):
    print(year,end=' ')
    ds_T = xr.open_dataset('/bettik/beaumetj/MARout/MAR-ERA-20C/daily/ICE.ERA-20C_v1.EUf.TTz.'+str(year)+'.nc')
    seasonal_mean_year=season_mean2(ds_T.TTz.sel(ztqlev=2)).sortby(xr.DataArray(['DJF','MAM','JJA', 'SON'],dims=['season']))
    if year> first_year:
        seasonal_mean2 += seasonal_mean_year
    else:
        seasonal_mean2 = xr.DataArray.copy(seasonal_mean_year)
seasonal_mean2 /= (last_year-first_year+1)

print(seasonal_mean.shape, seasonal_mean.season.data)

1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 (4, 126, 201) ['DJF' 'MAM' 'JJA' 'SON']


In [14]:
toto = seasonal_mean-seasonal_mean2
toto.max()

<xarray.DataArray ()>
array(0.03157058)
Coordinates:
    ztqlev   float32 2.0

In [15]:
toto.min()

<xarray.DataArray ()>
array(-0.0369893)
Coordinates:
    ztqlev   float32 2.0

In [9]:
toto

<xarray.DataArray (season: 4, y: 126, x: 201)>
array([[[ 0.00026554,  0.00032845,  0.00038117, ...,  0.0012556 ,
          0.00042049, -0.00046395],
        [-0.00044259, -0.00320693, -0.00402062, ...,  0.00406378,
          0.00253837, -0.00085439],
        [-0.00107146, -0.00456363, -0.00508967, ...,  0.00495958,
          0.00272795, -0.00126332],
        ...,
        [ 0.00083138, -0.00230362, -0.00483577, ..., -0.03425059,
         -0.03340041, -0.03132956],
        [ 0.00152543, -0.0005814 , -0.00236852, ..., -0.03385757,
         -0.0331268 , -0.03095215],
        [ 0.0022611 ,  0.00173018,  0.00133886, ..., -0.03067861,
         -0.03078831, -0.03076955]],

       [[ 0.00223791,  0.00227033,  0.00229202, ...,  0.00359863,
          0.00356425,  0.00351558],
        [ 0.00224336,  0.0024054 ,  0.00219035, ...,  0.00399609,
          0.00396547,  0.00348632],
        [ 0.00226288,  0.002189  ,  0.00192491, ...,  0.00408582,
          0.00399518,  0.00345297],
...
        [ 0.01370877,  0.01443854,  0.01404855, ...,  0.00986239,
          0.01034565,  0.00963904],
        [ 0.01384359,  0.01486446,  0.01460176, ...,  0.01010197,
          0.01032133,  0.00939978],
        [ 0.01398185,  0.01393127,  0.01388182, ...,  0.0093684 ,
          0.00933814,  0.00926357]],

       [[ 0.00148868,  0.00153806,  0.00155518, ...,  0.00390869,
          0.00396758,  0.00405253],
        [ 0.00145007,  0.00105805,  0.00117749, ...,  0.00341099,
          0.00360306,  0.00410819],
        [ 0.00141255,  0.0011748 ,  0.00133031, ...,  0.00327167,
          0.00352288,  0.00415211],
        ...,
        [ 0.00649366,  0.00629697,  0.00651522, ...,  0.00730505,
          0.00729545,  0.00802058],
        [ 0.00646456,  0.0064231 ,  0.00619637, ...,  0.00716942,
          0.00713105,  0.00804467],
        [ 0.00646268,  0.0065079 ,  0.00654891, ...,  0.00820066,
          0.00813373,  0.00805438]]])
Coordinates:
  * season   (season) object 'DJF' 'MAM' 'JJA' 'SON'
  * x        (x) float32 -483.0 -476.0 -469.0 -462.0 ... 896.0 903.0 910.0 917.0
  * y        (y) float32 -308.0 -301.0 -294.0 -287.0 ... 546.0 553.0 560.0 567.0
    ztqlev   float32 2.0